<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/kobert/tensorflow_Kobert_%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.4.2

     |████████████████████████████████| 2.0 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 64.8 MB/s 
     |████████████████████████████████| 3.3 MB 64.3 MB/s 


transformers라는 패키지는 허깅페이스라는 스타트업에서 만들었습니다.

In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [3]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import urllib.request
import logging
import tensorflow_addons as tfa

In [ ]:
logging.basicConfig(level=logging.ERROR)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/rsn_nlp_project/sample_0824.csv')

#라벨 데이터 category 정수로 변환

data_colname = 'contents'
label_colname = 'category'

encoder = LabelEncoder()
encoder.fit(dataset[label_colname])
dataset[label_colname] = encoder.transform(dataset[label_colname])


#이어서 contents 전처리(정제)
dataset[data_colname] = dataset[data_colname].str.replace("\(.*\)|\s-\s.*"," " ,regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("\[.*\]|\s-\s.*"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("\<.*\>|\s-\s.*"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("무단전재 및 재배포 금지"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("무단 전재 및 재배포 금지"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("©"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("ⓒ"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("저작권자"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace(".* 기자", " ", regex=True) #기자 이름에서 오는 유사도 차단
dataset[data_colname] = dataset[data_colname].str.replace("사진 = .*", " ", regex=True) #사진 첨부 문구 삭제
dataset[data_colname] = dataset[data_colname].str.replace("사진=.*", " ", regex=True) #사진 첨부 문구 삭제
dataset[data_colname] = dataset[data_colname].str.replace('\"', "",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)", " ", regex=True) #이메일 주소에서 오는 유사도 차단
dataset[data_colname] = dataset[data_colname].str.replace("\n"," ")
dataset[data_colname] = dataset[data_colname].str.replace("\r"," ")
dataset[data_colname] = dataset[data_colname].str.replace("\t"," ")
dataset[data_colname] = dataset[data_colname].str.replace( "\’" , "", regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("[ ]{2,}"," ",regex=True)

dataset.head()

,id,contents,category
0,NIRW2000000001.1,"목포시, '근대역사문화공간' 재생활성화 사업 흔들림 없이 추진 전남 목포시는 최근...",5
1,NIRW2000000001.2,대구·시흥 이어 안산도…홍역 전국 동시다발 '비상' 대구와 경기도 시흥에 이어 안...,4
2,NIRW2000000001.3,'확 달라진 베트남 축구' 박항서 매직은 현재 진행형 '박항서 매직'은 현재 진행...,6
3,NIRW2000000001.4,'홈' 창원에서 3년 만에 왕좌 되찾은 3점왕과 덩크왕 3점왕과 덩크왕이 홈코트에...,6
4,NIRW2000000001.5,대형사고 위험 77개 건설현장에 작업중지 명령 대형사고 위험이 높은 건설현장 77...,4


In [7]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 'IT/과학',
 1: '경제',
 2: '문화',
 3: '미용/건강',
 4: '사회',
 5: '생활',
 6: '스포츠',
 7: '연예',
 8: '정치'}

In [8]:
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

In [10]:
print('훈련용 데이터 개수 :',len(train)) # 훈련용 데이터 개수 출력

훈련용 데이터 개수 : 30221


In [11]:
print('테스트용 데이터 개수 :',len(test)) # 테스트용 데이터 개수 출력

테스트용 데이터 개수 : 7556


In [13]:
train.reset_index(drop=True, inplace=True) #중요!!!!
test.reset_index(drop=True, inplace=True) #중요!!!!

In [14]:
train[:5] # 상위 5개 출력

,id,contents,category
0,NIRW2000000001.17193,"'불펜·수비 말썽' 류현진, 온갖 불운에도 에이스 가치 증명 류현진 으로 호투했다...",6
1,NIRW2000000001.9678,10조 투자+α 프로젝트·인구정책 등 '10대 과제' 집중관리 정부가 대내외 악재...,1
2,NIRW2000000002.1485,"산업은행, '제2의 벤처붐' 조성 나선다 산업은행은 20일 여의도 본점에서 'KD...",1
3,NIRW2000000002.3289,주유소 휘발유 판매가 1350.3원…서울 1450.8원 3월 첫째주 주유소의 휘발...,5
4,NIRW2000000001.5480,제11대 박재규 경남대 총장 취임…대학을 새롭게 디자인 제11대 박재규 경남대학교...,4


In [15]:
test[:5] # 상위 5개 출력

,id,contents,category
0,NIRW2000000002.6699,"웰컴금융그룹, 어린이와 '행복나눔' 봉사활동 웰컴금융그룹 이 초록우산어린이재단과 ...",1
1,NIRW2000000001.16441,경남 우수 농식품 유럽 시장에 알렸다…6만 3천달러 판매 수익 경상남도가 도내 우...,4
2,NIRW2000000001.13146,"전남도, 여수 세계한상대회에서 청년 일자리 박람회 전라남도는 24일 여수엑스포 국...",4
3,NIRW2000000001.19289,"박영선 중기부장관, 구미 경제 현장 목소리 청취 박영선 중소벤처기업부 장관이 5일...",8
4,NIRW2000000001.946,충북선 철도 고속화 예타면제 확정 충북의 최대 숙원 사업으로 이른바 '강호축' 개...,5


Hugginface라는 스타트업의 transformers라는 패키지를 설치하시면  
모델 사용할 때는 항상 model.from_pretrained()

In [18]:
# 모델 로드
model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Koelectra 깃허브 : https://github.com/monologg/KoELECTRA

모델의 출력을 체크해봅시다.

In [31]:
max_seq_len = 128

In [25]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)

outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])

In [21]:
outputs

TFBaseModelOutputWithPooling([('last_hidden_state',
                               <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tf_bert_model')>),
                              ('pooler_output',
                               <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>)])

출력이 2개입니다.

outputs[0]은 모든 시점의 BERT의 최종 벡터. 문장 길이가 64일 경우에는 64 x 768  
outputs[1]은 CLS 시점의 BERT의 벡터

In [22]:
outputs[0]

<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tf_bert_model')>

In [23]:
outputs[1]

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>

In [26]:
# 토크나이저 코드 로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/monologg/KoBERT-Transformers/master/kobert_transformers/tokenization_kobert.py", filename="tokenization_kobert.py")

('tokenization_kobert.py', <http.client.HTTPMessage at 0x7f29cc3640d0>)

In [27]:
from tokenization_kobert import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

In [28]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[2, 2366, 5678, 5678, 1192, 1804, 6166, 5760, 3415, 4638, 3272, 3133, 6926, 3]


In [29]:
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['▁보는', '내', '내', '▁그대로', '▁들어', '맞', '는', '▁예측', '▁카리스마', '▁없는', '▁악', '역']


In [30]:
print(tokenizer.tokenize("전율을 일으키는 영화. 다시 보고싶은 영화"))

['▁전', '율을', '▁일으키', '는', '▁영화', '.', '▁다시', '▁보고', '싶', '은', '▁영화']


In [ ]:
print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))

[2, 4012, 7071, 3815, 5760, 3394, 54, 1574, 2358, 6751, 7086, 3394, 3]


In [ ]:
for elem in tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"):
  print(tokenizer.decode(elem))

[CLS]
전
율을
일으키
는
영화
.
다시
보고
싶
은
영화
[SEP]


In [ ]:
for elem in tokenizer.encode("happy birthday~!"):
  print(tokenizer.decode(elem))

[CLS]

ha
p
p
y

b
ir
th
d
ay
~
!
[SEP]


In [ ]:
print(tokenizer.decode(2))

[CLS]


In [ ]:
print(tokenizer.decode(3))

[SEP]


In [ ]:
max_seq_len=64

In [ ]:
print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=max_seq_len, pad_to_max_length=True))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[2, 4012, 7071, 3815, 5760, 3394, 54, 1574, 2358, 6751, 7086, 3394, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



토큰 인풋의 예를 들면 다음과 같습니다.  
문장을 토크나이징 하면 "전율을 일으키는 영화. 다시 보고싶은 영화"가
"'전', '##율', '##을', '일', '##으', '##키는', '영화', '.', '다시', '보고', '##싶', '##은', '영화'" 로 토크나이징이 됩니다.  
이거를 버트 인풋에 들어갈 숫자로 바꿔주면,  
["2, 4012, 7071, 3815, 5760, 3394, 54, 1574, 2358, 6751, 7086, 3394, 3"]  
로 바뀌게 됩니다. 여기 나오는 숫자들이 버트 인풋에 들어가는 토큰 인풋입니다.
버트 모형에 들어가는 인풋은 사실 일정한 길이를 가져야 합니다.(본 예제에서는 64)
따라서 남는 부분은 1로 채워지게 됩니다(패딩)

In [ ]:
# 세그멘트 인풋
print([0]*64)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


세그멘트 인풋은 문장이 앞문장인지 뒷문장인지 구분해주는 역할을 하는데요 본 문장에서는 문장 하나만 인풋으로 들어가기 때문에 0만 들어가게 되고, 문장 길이만큼의 0이 인풋으로 들어가게 됩니다.

In [ ]:
# 마스크 인풋
valid_num = len(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))
print(valid_num * [1] + (max_seq_len - valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


숫자 2 : [CLS]  
숫자 3 : [SEP]


어텐션 마스크 인풋은 토큰 인풋에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 두게 됩니다.  

종합하면,
버트의 인풋은 토큰, 세그먼트, 마스크로 이루어져 있습니다.
"전율을 일으키는 영화. 다시 보고싶은 영화" 라는 문장을 가지고 예를 들면,  

토큰 인풋 : 
[2, 4012, 7071, 3815, 5760, 3394, 54, 1574, 2358, 6751, 7086, 3394, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

세그먼트 인풋 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

어텐션 마스크 인풋 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  

네이버 영화 평가 문장들을 버트 인풋으로 바꿔보도록 하겠습니다.
문장이 토큰 인풋, 세그먼트 인풋, 마스크 인풋으로 변환 됩니다.
huggingface에서는 순서가 [토큰 인풋, 마스크 인풋, 세그먼트 인풋] 입니다.

In [67]:
from tensorflow.keras.utils import to_categorical
NUM_CLASS = 9

def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드 임베딩을 위한 문장의 정수 인코딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
       
        # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        
        # token_type_id는 세그먼트 임베딩을 위한 것으로 이번 예제는 문장이 1개이므로 전부 0으로 통일.
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label = to_categorical(label, num_classes=NUM_CLASS, dtype=np.int32)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [33]:
max_seq_len = 128

In [68]:
train_X, train_y = convert_examples_to_features(train['contents'], train['category'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/30221 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 30221/30221 [00:51<00:00, 586.82it/s]


In [69]:
test_X, test_y = convert_examples_to_features(test['contents'], test['category'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/7556 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 7556/7556 [00:13<00:00, 556.53it/s]


In [70]:
# 최대 길이: 128
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [   2  520 6424 7716  478 6629 6441 1958 6768   15 1898   46 3437 5353
 2485 7010 6901 3290  771 4293 1898  517 7078 5096 7869   54 1581 5330
  589 5808   93 6079 3188  757 1915 6903 1442 5348 7330 1581 2485 7716
 7096  621 7953 6983  624 7953 6896  774  553 7220 6792 1434 7306   54
 1899  533 6703 1308 1522 7444 5439 4797 7086  589 5808  172 6079  517
 7250   54  517 5540 5859 1899 2880 2497 7078 2952 1522 7489 4525  553
 6328 7095 5105 6903 1370 6117 7095 2828 7088  517 7415 5423   54 2880
 1406 7253 1849 4765 5330 2095 5691 7350 2872 3860 2691 3036 7223 4532
 7810 3913  971 7088 3848 7869   54 1210 1582 1898 7095 2952 4344 7327
 2086    3]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
세그먼트 인코딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [71]:
train_X[0].shape, train_X[1].shape, train_X[2].shape, train_y.shape

((30221, 128), (30221, 128), (30221, 128), (30221, 9))

TPU 사용법 : https://wikidocs.net/119990

In [38]:
# TPU 작동을 위한 코드 TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.26.107.26:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.26.107.26:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [90]:
num_epochs = 5
num_batch = 16

In [91]:
warmup_ratio = 0.1

t_total = len(train) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [92]:
LR = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=0.0001, first_decay_steps=warmup_step)

In [93]:
initializer_range = 0.2

In [94]:
def create_model():
  model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)

  input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
  attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
  token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)

  outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])
  pooled_output = outputs[1]

  optimizer = tfa.optimizers.RectifiedAdam(learning_rate=LR)
  
  pooled_output = tf.keras.layers.Dropout(0.5)(pooled_output)
  prediction = tf.keras.layers.Dense(9, activation='softmax', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=initializer_range))(pooled_output)
  cls_model = tf.keras.Model([input_ids_layer, attention_masks_layer, token_type_ids_layer], prediction)
  cls_model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])
  cls_model.summary()

  return cls_model

In [95]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
####
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('/content/drive/MyDrive/result_cp0824/checkpoint_epoch8', monitor='accuracy', mode='max', verbose=1, save_weights_only=True, save_best_only=True)
####
strategy = tf.distribute.experimental.TPUStrategy(resolver)
# TPU를 활용하기 위해 context로 묶어주기
with strategy.scope():
  cls_model = create_model()
  cls_model.fit(train_X, train_y, callbacks=[es, mc], epochs=num_epochs, batch_size=num_batch, validation_data=(test_X, test_y))

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 9) dtype=int32>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 9) dtype=int32>]


1889/1889 [==============================] - ETA: 0s - loss: 0.9202 - accuracy: 0.7364

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 9) dtype=int32>]


1889/1889 [==============================] - 264s 66ms/step - loss: 0.9202 - accuracy: 0.7364 - val_loss: 0.6590 - val_accuracy: 0.8007

Epoch 00001: accuracy improved from -inf to 0.73641, saving model to /content/drive/MyDrive/result_cp0824/checkpoint_epoch8
Epoch 2/5
1889/1889 [==============================] - 115s 61ms/step - loss: 0.5755 - accuracy: 0.8238 - val_loss: 0.5545 - val_accuracy: 0.8264

Epoch 00002: accuracy improved from 0.73641 to 0.82380, saving model to /content/drive/MyDrive/result_cp0824/checkpoint_epoch8
Epoch 3/5
1889/1889 [==============================] - 116s 61ms/step - loss: 0.4476 - accuracy: 0.8605 - val_loss: 0.5688 - val_accuracy: 0.8395

Epoch 00003: accuracy improved from 0.82380 to 0.86046, saving model to /content/drive/MyDrive/result_cp0824/checkpoint_epoch8
Epoch 4/5
1889/1889 [==============================] - 115s 61ms/step - loss: 0.3302 - accuracy: 0.8965 - val_loss: 0.5956 - val_accuracy: 0.8294

Epoch 00004: accuracy improved from 0.86046 

In [96]:
results = cls_model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)

8/8 [==============================] - 17s 1s/step - loss: 0.6030 - accuracy: 0.8515
test loss, test acc:  [0.6029975414276123, 0.8515087366104126]


In [108]:
def sentence_prediction(example):
    global tokenizer

    input_ids, attention_masks, token_type_ids = [], [], []

    input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
       
    # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        
    # token_type_id는 세그먼트 임베딩을 위한 것으로 이번 예제는 문장이 1개이므로 전부 0으로 통일.
    token_type_id = [0] * max_seq_len
  
    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)

    input_ids = np.array(input_ids)
    attention_masks = np.array(attention_masks)
    token_type_ids = np.array(token_type_ids)
    return [input_ids, attention_masks, token_type_ids]

def evaluation_predict(sentence):
    data_x = sentence_prediction(sentence)
    predict = cls_model.predict(data_x)

    print('예측 결과 수치', predict)
    print(predict)
    predict_answer = np.argmax(predict[0])
    predict_value = predict[0][predict_answer]
    
    if predict_answer == 0:
      print("(IT/과학 확률 : %.2f) IT/과학 뉴스입니다." % (1-predict_value))
      return 0
    elif predict_answer == 1:
      print("(경제 확률 : %.2f) 경제 뉴스입니다." % predict_value)
      return 1
    elif predict_answer == 2:
      print("(문화 확률 : %.2f) 문화 뉴스입니다." % predict_value)
      return 2
    elif predict_answer == 3:
      print("(미용/건강 확률 : %.2f) 미용/건강 뉴스입니다." % predict_value)
      return 3
    elif predict_answer == 4:
      print("(사회 확률 : %.2f) 사회 뉴스입니다." % predict_value)
      return 4
    elif predict_answer == 5:
      print("(생활 확률 : %.2f) 생활 뉴스입니다." % predict_value)
      return 5
    elif predict_answer == 6:
      print("(스포츠 확률 : %.2f) 스포츠 뉴스입니다." % predict_value)
      return 6
    elif predict_answer == 7:
      print("(연예 확률 : %.2f) 연예 뉴스입니다." % predict_value)
      return 7
    elif predict_answer == 8:
      print("(정치 확률 : %.2f) 정치 뉴스입니다." % predict_value)
      return 8

In [109]:
text = """
진중권 전 동양대 교수는 이준석 국민의힘 대표에게 국민권익위원회가 발표한 부동산 거래내역 전수조사 결과 국민의힘 소속 의원들의 엄정한 조치를 요구했습니다. 이를 잘 해결할 경우 ‘리더십 리스크’를 해결할 수 있을 것이라 내다봤습니다.
진 전 교수는 지난 23일 CBS라디오 ‘한판 승부’에 출연해 국민권익위 전수조사 결과 국민의힘 의원 12명에 대해 투기 의혹이 제기된 것에 대해 “대선에 큰 변수가 될 것 같지 않다”고 전망했습니다.
이어 “경찰수사, 검찰 수사에서 무혐의가 나올 것들이 상당히 많고 진짜 투기를 한 사람이 얼마나 되는지는 좀 더 지켜봐야” 한다며 “국민의힘에서 어떻게 처리할 것인가 더 큰 문제”라고 했습니다."""
evaluation_predict(text)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


예측 결과 수치 [[3.9573982e-03 1.9504441e-02 3.5049228e-04 1.1931474e-03 3.7831072e-02
  9.2827511e-04 6.2081649e-04 2.1891363e-04 9.3539554e-01]]
[[3.9573982e-03 1.9504441e-02 3.5049228e-04 1.1931474e-03 3.7831072e-02
  9.2827511e-04 6.2081649e-04 2.1891363e-04 9.3539554e-01]]
(정치 확률 : 0.94) 정치 뉴스입니다.


8

In [ ]:
predicted_data = []
for x in test['contents']:
  predicted_data.append(evaluation_predict(x))

In [2]:
from sklearn.metrics import classification_report
print(classification_report(test_y, predicted_data, target_names=['부정', '긍정']))

NameError: ignored

In [ ]:
movie_evaluation_predict("스토리는 확실히 실망이였지만 배우들 연기력이 대박이였다 특히 이제훈 연기 정말 ... 이 배우들로 이렇게밖에 만들지 못한 영화는 아쉽지만 배우들 연기력과 사운드는 정말 빛났던 영화. 기대하고 극장에서 보면 많이 실망했겠지만 평점보고 기대없이 집에서 편하게 보면 괜찮아요. 이제훈님 연기력은 최고인 것 같습니다")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


예측 결과 수치 [[0.9998368]]
0.5 기준으로 레이블 변환 1.0
(긍정 확률 : 1.00) 긍정적인 영화 평가입니다.


In [ ]:
movie_evaluation_predict("남친이 이 영화를 보고 헤어지자고한 영화. 자유롭게 살고 싶다고 한다. 내가 무슨 나비를 잡은 덫마냥 나에겐 다시 보고싶지 않은 영화.")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


예측 결과 수치 [[0.0114176]]
0.5 기준으로 레이블 변환 0.0
(부정 확률 : 0.99) 부정적인 영화 평가입니다.


In [ ]:
movie_evaluation_predict("이 영화 존잼입니다 대박")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


예측 결과 수치 [[0.9909471]]
0.5 기준으로 레이블 변환 1.0
(긍정 확률 : 0.99) 긍정적인 영화 평가입니다.


In [ ]:
np.array([test_X[0][0]]), np.array([test_X[0][1]]), np.array([test_X[0][2]])

(array([[   2,  517, 5515,  517,  492,    3,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1]]),
 array([[  2, 650, 278, 322, 351, 329, 334, 311, 264, 349, 302, 351, 294,
         282, 274, 311, 356, 270,   3,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1]]),
 array([[   2, 2145, 6844, 3647, 4841, 7220, 5937,   54,   55, 1370, 6488,
         7344, 3165,  533, 7220,  517, 7362, 5760, 1698, 5837, 7009, 3097,
         7173, 6797,    3,    1,    1,   

In [ ]:
for word in test_X[0][0]:
  print(tokenizer.decode([word]))

[CLS]

굳

ᄏ
[SEP]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
